In [1]:
# We give an example notebook on how to extract weight vector via NMF and compute inner product

In [8]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.decomposition import NMF
import torch.nn as nn
import cv2
import numpy as np
from torchvision.utils import save_image
from tqdm import tqdm
import matplotlib.image 
import numpy as np
import cv2
from mt_model_gpu_grad import MtModel
from kernels_gpu_grad import *
model1 = torch.load('2023_04_08 04_33 FOLD 0 MTMK L1.pt')
model1 = model1.to('cuda')
for kernel in model1.kernel_list:
    kernel.kernel = kernel.kernel.cpu()

In [9]:
# we use heart, which corresponds to index 2
alpha = model1.alpha[2].cpu()
X_train = model1.X_train[2].cpu()
Y_train= model1.Y_train[2].cpu()
phecodes = list(model1.train_phecode[2])

In [10]:
# get weight vector
# we empirically find that alpha is parallel to y, so we directly use abs instead of element-wise division as written in the paper
model = NMF(n_components=1, init='random', random_state=53)
wv = model.fit_transform(torch.abs(alpha).numpy())
wv = torch.from_numpy(wv).float()

In [11]:
wv

tensor([[0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [1.0332e+00],
        [1.3464e-02],
        [1.1906e-06],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00],
        [0.0000e+00]])

In [12]:
Kernel_matrices = []
for i in range(len(model1.kernel_list)):
    Kernel_matrices.append(model1.kernel_list[i](X_train, X_train))
    
Kernel_matrix = torch.zeros_like(Kernel_matrices[0])
for i in range(len(model1.kernel_list)):
    Kernel_matrix += wv[i] * Kernel_matrices[i]

In [13]:
Kernel_matrix

tensor([[ 1.0467e+00,  7.5282e-01,  7.0903e-01,  ...,  4.5829e-01,
          5.8240e-01,  7.6881e-02],
        [ 7.5282e-01,  1.0467e+00,  8.7165e-01,  ...,  9.3803e-02,
          2.0256e-01,  4.0072e-01],
        [ 7.0903e-01,  8.7165e-01,  1.0467e+00,  ..., -6.4409e-01,
         -4.5183e-01, -1.8158e-01],
        ...,
        [ 4.5829e-01,  9.3803e-02, -6.4409e-01,  ...,  9.3829e+02,
          6.3643e+02,  1.8761e+02],
        [ 5.8240e-01,  2.0256e-01, -4.5183e-01,  ...,  6.3643e+02,
          7.8964e+02,  1.5946e+02],
        [ 7.6881e-02,  4.0072e-01, -1.8158e-01,  ...,  1.8761e+02,
          1.5946e+02,  2.1246e+02]], dtype=torch.float64,
       grad_fn=<AddBackward0>)